## Huggingface dataset version of childes

In [4]:
import os
from os.path import join, exists

from transformers import BertForMaskedLM, BertTokenizer
import datasets
from datasets import load_dataset

import importlib

## Tests and verifications

In [5]:
import transformers

from transformers import BertForMaskedLM, BertTokenizer

import model_finetuning
importlib.reload(model_finetuning)

this_model, this_tokenizer = model_finetuning.get_bert_model()
print('modules retrieved')
maintain = model_finetuning.get_datasets(this_tokenizer, with_tags = True)
print('first dataset')
remove = model_finetuning.get_datasets(this_tokenizer, with_tags = False)
print('second dataset')


Some weights of the model checkpoint at bert-large-uncased-whole-word-masking were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


modules retrieved


Dataset childes downloaded and prepared to ./user/wongn/temp_cache3/childes/childes_data/1.0.0/f8e634d7b7b5f2a90912db30fd2fa4d0cd911ac7fae250c1adeaedfcd8444872. Subsequent calls will reuse this data.



first dataset


Reusing dataset childes (./user/wongn/temp_cache3/childes/childes_data/1.0.0/f8e634d7b7b5f2a90912db30fd2fa4d0cd911ac7fae250c1adeaedfcd8444872)



second dataset


In [8]:

chi_test_idx = 0
cgv_test_idx = 628

run_maintain_idx = lambda test_idx : this_tokenizer.convert_ids_to_tokens(maintain['validation'][test_idx]['input_ids'])
run_remove_idx = lambda test_idx : this_tokenizer.convert_ids_to_tokens(remove['validation'][test_idx]['input_ids'])


print('These should have chi and cgv as single tokens.')
print(run_maintain_idx(chi_test_idx))
print(run_maintain_idx(cgv_test_idx))

print('These should not have chi and cgv inside.')
print(run_remove_idx(chi_test_idx))
print(run_remove_idx(cgv_test_idx))

# Verified 6/18 with fresh cache download!


These should have chi and cgv as single tokens.
['[CLS]', '[chi]', 'i', "'", 'm', 'not', 'going', 'to', 'do', 'anything', '.', '[SEP]']
['[CLS]', '[cgv]', 'back', 'on', 'the', 'table', 'if', 'you', 'wanna', 'finish', 'it', '.', '[SEP]']
These should not have chi and cgv inside.
['[CLS]', 'i', "'", 'm', 'not', 'going', 'to', 'do', 'anything', '.', '[SEP]']
['[CLS]', 'back', 'on', 'the', 'table', 'if', 'you', 'wanna', 'finish', 'it', '.', '[SEP]']
